In [46]:
import pandas as pd
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
INPUT_DATA_FOLDER = ""
DATA_FILE = "all_days_isw_reports.csv"

OUTPUT_FOLDER = "2_isw_parsed"
OUTPUT_DATA_FILE = "all_days_isw_reports_parsed.csv"


C:\Users\slaye\AppData\Local\Temp\ipykernel_10184\295368298.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [47]:
#df = pd.read_csv(f"{INPUT_DATA_FOLDER}/{DATA FILE)", sep=";")
df = pd.read_csv(f"{DATA_FILE}", sep=";")

In [48]:
df.head(10)

,date,title,text_title:,full_url,main_html
0,2022-11-03,"Russian Offensive Campaign Assessment, Novembe...","Russian Offensive Campaign Assessment, November 3",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t..."


In [49]:
test_row = df.iloc[0]
page_html_text = test_row["main_html"]
print(page_html_text)

<div class="field field-name-body field-type-text-with-summary field-label-hidden"><div class="field-items"><div class="field-item even" property="content:encoded"> <p><strong><br/></strong></p>
<p><strong>Russian Offensive Campaign Assessment, November 3</strong></p>
<p><strong>Kateryna Stepanenko, Riley Bailey, Grace Mappes, Yekaterina Klepanchuk, and Frederick W. Kagan</strong></p>
<p><strong>November 3, 9:15 pm ET</strong></p>
<p><strong>Click </strong><a href="https://storymaps.arcgis.com/stories/36a7f6a6f5a9448496de641cf64bd375"><strong>here</strong></a><strong> to see ISW’s interactive map of the Russian invasion of Ukraine. This map is updated daily alongside the static maps present in this report.</strong></p>
<p><strong>Russian forces are continuing to withdraw some elements from northwestern Kherson Oblast, but it is still unclear if Russian forces will fight for Kherson City. </strong>Kherson City occupation deputy Kirill Stremousov stated on November 3 that Russian forces 

In [50]:
display(HTML(page_html_text))

In [80]:
def remove_names_and_date(page_html_text):
    parsed_html = BeautifulSoup(page_html_text)
    p_lines = parsed_html.findAll('p')
    
    min_sentense_word_count = 13
    p_index = 0
    
    #find first long sentense
    for p_line in p_lines:
        
        strong_lines = p_line.findAll('strong')
        if not strong_lines:
            continue

        for s in strong_lines:
            if len(s.text.split(" ")) >= min_sentense_word_count:
                break
        else:
            p_index += 1
            continue
        break
    for i in range(0, p_index):
        page_html_text = page_html_text.replace(str(p_lines[i]), "")
        
    return page_html_text   

In [81]:
df['main_html_v2'] = df['main_html'].apply(lambda x: remove_names_and_date(x))

In [82]:

display(HTML(df.iloc[0]["main_html_v2"]))

In [87]:
import re
pattern = "\[(\d+)\]"
df['main_html_v3'] = df['main_html_v2'].apply(lambda x: re.sub(pattern, "", x))
#display(HTML(df.iloc[0]["main_html_v3"]))

In [90]:
df['main_html_v4'] = df['main_html_v3'].apply(lambda x: BeautifulSoup(x).text)
#display(HTML(df.iloc[0]["main_html_v4"]))

In [91]:
df['main_html_v5'] = df['main_html_v4'].apply(lambda x: re.sub(r'http(\S+.*\s)', "", x))
df['main_html_v6'] = df['main_html_v5'].apply(lambda x: re.sub(r'(©2022|©2023|2022|2023)', "", x))

In [92]:
display(HTML(df.iloc[0]["main_html_v6"]))